**The same task as Lab 1 except using `RNNs`**

**Watch out!<br>
Preproceesing here is a a bit different**

**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets. <br>
0 -> no hate speech <br>
1 -> contains hate speech <br>

**Total Estimated Time = 60-90 Mins**

> **Load the `clean data` preprocessed in `Lab 1`, then handle it to be used with `RNNs`**

### Import Libraries

In [20]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

### Load Dataset

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
df.head()
#print(df.shape)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


<h1>
    EDA

In [4]:
#check missing values
df.isna().sum()

id       0
label    0
tweet    0
dtype: int64

In [6]:
#check duplicates 
df.duplicated().sum()

0

### Preprocessing

In [7]:
#drop id 
df.drop(columns = 'id' , inplace = True)

In [8]:
#remove duplicates 
df = df.drop_duplicates()

In [9]:
df.duplicated().sum()

0

In [10]:
# relation between # , :) and label !
# problems : #  @user  emojii  punctuate  url_validate 
# want to check if sentenses had hate speech or not 

In [11]:
def remove_at(txt):
    clean_txt = re.sub('@user', '',txt )
    return clean_txt
#checked 

In [12]:
def remove_pun(txt):
    regex = r"[!\"\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
    subst = ""
    clean_txt = re.sub(regex, subst, txt, 0, re.MULTILINE)
    return clean_txt
#checked

In [13]:
def remove_pun(txt):
    regex = r"[!\"\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
    subst = ""
    clean_txt = re.sub(regex, subst, txt, 0, re.MULTILINE)
    return clean_txt
#checked 

In [14]:
def remove_non_english(txt):
    clean_txt = re.sub(r"[^a-zA-Z0-9#]", ' ',txt )
    return clean_txt
#checked  

In [15]:
def contains_url(text):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return bool(pattern.search(text))

In [16]:
df['tweet'].apply(contains_url)

0        False
1        False
2        False
3        False
4        False
         ...  
31956    False
31957    False
31958    False
31959    False
31961    False
Name: tweet, Length: 29530, dtype: bool

In [17]:
def wrangle(txt):
    clean_txt1 = remove_at(txt)
    clean_txt2 = remove_pun(clean_txt1)
    clean_txt3 = remove_non_english(clean_txt2)
    clean_txt4 = clean_txt3.lower()
    return clean_txt4
    

In [18]:
df['tweet'] = df['tweet'].apply(wrangle)

### Modelling

<h1>
    Split Data


In [24]:
print("Unique words in corpus:", df['tweet'].nunique())
print("Unique tags in corpus:", df['label'].nunique())

Unique words in corpus: 29486
Unique tags in corpus: 2


In [25]:
tweets = list(set(df['tweet'].values))
#words.append("ENDPAD")
num_tweets = len(tweets)
num_tweets

29486

In [27]:
labels = list(set(df['label'].values))
num_labels = len(labels)
num_labels

2

In [28]:
word2idx = {w: i + 1 for i, w in enumerate(tweets)}


In [33]:
target = 'label'
feature = 'tweet'
X = df[feature]
y = df[target]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
   X , y, test_size=0.33, random_state=42)

In [35]:
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [36]:
# Tokenize our training data
tokenizer = Tokenizer(num_words=num_tweets, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(X_train)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)


In [37]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)


In [39]:
VOCAB_SIZE = len(word2idx)
VOCAB_SIZE

29486

In [47]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [48]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.Precision(thresholds=0)])

In [50]:
history = model.fit(train_padded, y_train, epochs=10, batch_size=128,
                    validation_data=(test_padded, y_test), 
                    validation_steps=30)

Epoch 1/10
155/155 [==============================] - 6s 40ms/step - loss: 0.0444 - precision: 0.0679 - val_loss: 0.2667 - val_precision: 0.0695
Epoch 2/10
155/155 [==============================] - 6s 39ms/step - loss: 0.0176 - precision: 0.0679 - val_loss: 0.3043 - val_precision: 0.0695
Epoch 3/10
155/155 [==============================] - 6s 39ms/step - loss: 0.0144 - precision: 0.0679 - val_loss: 0.3353 - val_precision: 0.0695
Epoch 4/10
155/155 [==============================] - 6s 38ms/step - loss: 0.0073 - precision: 0.0679 - val_loss: 0.3406 - val_precision: 0.0695
Epoch 5/10
155/155 [==============================] - 6s 41ms/step - loss: 0.0054 - precision: 0.0679 - val_loss: 0.3866 - val_precision: 0.0695
Epoch 6/10
155/155 [==============================] - 6s 40ms/step - loss: 0.0066 - precision: 0.0679 - val_loss: 0.5115 - val_precision: 0.0695
Epoch 7/10
155/155 [==============================] - 6s 39ms/step - loss: 0.0084 - precision: 0.0679 - val_loss: 0.3569 - val_pre

#### Evaluation

In [52]:
test_loss, test_per = model.evaluate(test_padded, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Percision: {}'.format(test_per))

305/305 [==============================] - 1s 4ms/step - loss: 0.3932 - precision: 0.0688
Test Loss: 0.3932141065597534
Test Percision: 0.0687532052397728


### Enhancement

In [53]:
# using LSTM
lstm_bi = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [54]:
lstm_bi.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.Precision(thresholds=0)])

In [57]:
history = lstm_bi.fit(train_padded, y_train, epochs=10,
                    validation_data=(test_padded, y_test), 
                    validation_steps=30)

Epoch 1/10
619/619 [==============================] - 33s 53ms/step - loss: 0.0029 - precision_1: 0.0679 - val_loss: 0.2884 - val_precision_1: 0.0688
Epoch 2/10
619/619 [==============================] - 35s 57ms/step - loss: 0.0041 - precision_1: 0.0679 - val_loss: 0.2962 - val_precision_1: 0.0688
Epoch 3/10
619/619 [==============================] - 37s 60ms/step - loss: 0.0020 - precision_1: 0.0679 - val_loss: 0.2663 - val_precision_1: 0.0688
Epoch 4/10
619/619 [==============================] - 34s 55ms/step - loss: 6.8451e-04 - precision_1: 0.0679 - val_loss: 0.3854 - val_precision_1: 0.0688
Epoch 5/10
619/619 [==============================] - 38s 61ms/step - loss: 5.0617e-04 - precision_1: 0.0679 - val_loss: 0.4081 - val_precision_1: 0.0688
Epoch 6/10
619/619 [==============================] - 34s 55ms/step - loss: 3.3562e-04 - precision_1: 0.0679 - val_loss: 0.3849 - val_precision_1: 0.0688
Epoch 7/10
619/619 [==============================] - 34s 55ms/step - loss: 2.1745e-04 -

In [58]:
test_loss, test_per = lstm_bi.evaluate(test_padded, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Percision: {}'.format(test_per))

305/305 [==============================] - 3s 11ms/step - loss: 0.5623 - precision_1: 0.0688
Test Loss: 0.5622667670249939
Test Percision: 0.0687532052397728


### Results & Conclusion

In [ ]:
# I have noticed the difference between on simple rnn and lstm  on accuracy but not percision (i used the percision cause the unbalanced data but i'm not sure) ?!


#### Done!